# 대규모 데이터 학습

** 정의 **
- 빅데이터는 메모리 등의 문제로 특정한 모형은 사용할 수 없는 경우가 있다.
- 이러한 경우 다음의 모형을 사용하여 전체 데이터를 조각으로 나누어 학습하는 점진적 학습 방법을 사용할수 있다.  
   - 사전확률분포를 설정할 수 있는 생성 모형
   - 시작 가중치를 설정할 수 있는 모형
- 모형의 특성에 따라서 partial_fit()의 기능이 조금씩 다르다.
- SGDClassifier() : 확률적 경사하강법 모형
   - model.partial_fit(X, y, classes=classes)
   - partial_fit() 을 사용하여 경사하강법을 단계적으로 시행한 후 다음 단계에서 가중치를 사용할 수 있다. 
   - 1 epoch 당 10번의 split 데이터를 학습하여 누적된 모형의 점수를 반환할 수 있다.
- NB
   - clf.partial_fit(X, y, classes=classes) : 1 batch sample을 학습한 확률분포를 다음 훈련 모형의 사전확률분포로 사용할 수 있다. 
   - SGDC와 방식이 다르다.
- 그래디언트 부스팅 : LightGBM
   - lightgbm을 사용하면 초기 커미티 멤버로 일부 데이터를 학습한 모형을 사용할 수 있다.
   - lightgbm의 train() 클래스를 사용한다. 특정 모형을 구현하지 않고, train() 클래스에서 params와 데이터 등을 정의 한다.
   - 내장 된 모형이 있는 것 같다.
- RF ensemble
   - RF 등의 ensemble 모형에서는 일부 데이터를 사용한 모형을 개별 분류기로 사용할 수 있다. 
   - clf.fit(X, y)
   - clf.n_estimators += n_tree_step
- 빅데이터의 예측 모형은 메모리 등 컴퓨터 자원을 효율적으로 활용하기 위하여 학습 데이터를 조각하여 훈련한다.
   - 모형의 매서드로 모형을 누적하여 fitting 해준다. 누적 fitting 방식은 모형마다 다르다.
   - SGDClassifier
   - NB
   - Gradient Boosting
   - RF
- 데이터를 어떻게 분할하여 학습하고 지표를 반환할 것인지 알고리즘 적 판단을 해야할 것 같다.
   

In [1]:
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
covtype = fetch_covtype(shuffle=True, random_state=0)
X_covtype = covtype.data
y_covtype = covtype.target

X_covtype.shape, y_covtype.shape

((581012, 54), (581012,))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_covtype, y_covtype)

In [6]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((435759, 54), (145253, 54), (435759,), (145253,))

In [9]:
classes = np.unique(y_covtype)
classes

array([1, 2, 3, 4, 5, 6, 7])

In [20]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
X_train.mean()

0.12127056091780815

In [16]:
def read_Xy(start, end) : 
    
    # strat ~ y_train 길이와 end 중 작은값 까지
    idx = list(range(start, min(len(y_train) -1, end)))
    X = X_train[idx, :]
    y = y_train[idx]
    
    return X, y

## SGD 사용
- 퍼셉트론 모형은 가중치를 계속 업데이트한다.
- 일부 데이터를 사용하여 가중치를 구하고, 다음 단계에서 초기 가중치로 사용가능
- 1 epoch 당 10번의 split 학습 데이터를 fitting 하고 score 계산

In [31]:
%%time

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

model = SGDClassifier(random_state=0)
n_split = 10
n_X = len(y_train) // n_split
n_epoch = 10
for epoch in range(n_epoch) : 
    for n in range(n_split) : 
        ## start : 0~9 순차적으로 n_X와 곱해진다.
        start = n * n_X  
        ## end : 1~10 순차적으로 n_X와 곱해진다.
        end = (n+1) * n_X  
        ## start, end로 만든 idx로 X, y 반환
        X, y = read_Xy(start, end)
        ## partial_fit : one epoch 경사하강법 수행 
        model.partial_fit(X, y, classes=classes)
    accuracy_train = accuracy_score(y_train, model.predict(X_train))
    accuracy_test = accuracy_score(y_test, model.predict(X_test))
    print("epoch={:d} train acc={:5.3f} test acc={:5.3f}"\
          .format(epoch, accuracy_train, accuracy_test))

epoch=0 train acc=0.705 test acc=0.706
epoch=1 train acc=0.705 test acc=0.705
epoch=2 train acc=0.705 test acc=0.705
epoch=3 train acc=0.705 test acc=0.705
epoch=4 train acc=0.707 test acc=0.707
epoch=5 train acc=0.707 test acc=0.708
epoch=6 train acc=0.707 test acc=0.708
epoch=7 train acc=0.708 test acc=0.708
epoch=8 train acc=0.708 test acc=0.709
epoch=9 train acc=0.709 test acc=0.709
CPU times: total: 15.3 s
Wall time: 8.51 s


In [30]:
n_split = 10
n_X = len(y_train) // n_split
for n in range(n_split) : 
    start = n * n_X
    end = (n+1) * n_X
    print(n, n+1, start, end)

0 1 0 43575
1 2 43575 87150
2 3 87150 130725
3 4 130725 174300
4 5 174300 217875
5 6 217875 261450
6 7 261450 305025
7 8 305025 348600
8 9 348600 392175
9 10 392175 435750


## Naive Bayes 
- clf.partial_fit(X, y, classes=classes)
   - batch 사이즈의 샘플에 대해 증분 fitting 한다.

In [32]:
%%time

from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()

n_split = 10
n_X = len(y_train) // n_split
for n in range(n_split) : 
    start = n * n_X
    end = (n+1) * n_X
    X, y = read_Xy(start, end)
    model.partial_fit(X, y, classes=classes)
    acc_train = accuracy_score(y_train, model.predict(X_train))
    acc_test = accuracy_score(y_test, model.predict(X_test))
    print("n={:d} train acc={:5.3f} test acc={:5.3f}".format(n, acc_train, acc_test))

n=0 train acc=0.635 test acc=0.635
n=1 train acc=0.634 test acc=0.634
n=2 train acc=0.632 test acc=0.632
n=3 train acc=0.632 test acc=0.632
n=4 train acc=0.633 test acc=0.633
n=5 train acc=0.632 test acc=0.633
n=6 train acc=0.632 test acc=0.632
n=7 train acc=0.632 test acc=0.632
n=8 train acc=0.631 test acc=0.632
n=9 train acc=0.632 test acc=0.632
CPU times: total: 15 s
Wall time: 3.84 s


In [35]:
%%time

from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()

n_split = 10
n_X = len(y_train) // n_split
n_epoch = 10
for epoch in range(n_epoch) : 
    for n in range(n_split) : 
        start = n * n_X
        end = (n+1) * n_X
        X, y = read_Xy(start, end)
        model.partial_fit(X, y, classes=classes)
    acc_train = accuracy_score(y_train, model.predict(X_train))
    acc_test = accuracy_score(y_test, model.predict(X_test))
    print("epoch={:d} train acc={:5.3f} test acc={:5.3f}".format(epoch, acc_train, acc_test))

epoch=0 train acc=0.632 test acc=0.632
epoch=1 train acc=0.632 test acc=0.632
epoch=2 train acc=0.632 test acc=0.632
epoch=3 train acc=0.632 test acc=0.632
epoch=4 train acc=0.632 test acc=0.632
epoch=5 train acc=0.632 test acc=0.632
epoch=6 train acc=0.632 test acc=0.632
epoch=7 train acc=0.632 test acc=0.632
epoch=8 train acc=0.632 test acc=0.632
epoch=9 train acc=0.632 test acc=0.632
CPU times: total: 28.9 s
Wall time: 7.95 s


## Gradient Boosting
- 초기 커미티 멤버로 일부 데이터를 학습한 모형을 사용할 수 있다.

In [38]:
!conda list "^light"

# packages in environment at C:\DS\Anaconda3\envs\dev_env:
#
# Name                    Version                   Build  Channel
lightgbm                  4.1.0           py310h5da7b33_0  


In [52]:
%%time

from lightgbm import train, Dataset

params = {
    "objective": "multiclass",
    "num_class": len(classes)+1,
    "learning_rate": 0.2,
    "seed": 0
}

n_split = 10
n_X = len(y_train) // n_split
num_tree = 10
model = None
for n in range(n_split) : 
    start = n * n_X
    end = (n+1) * n_X
    
    X, y = read_Xy(start, end)
    model = train(params, 
                  init_model=model, 
                  train_set=Dataset(X, y),
                  keep_training_booster=False, 
                  num_boost_round=num_tree)
    
    acc_train = accuracy_score(y_train, np.argmax(model.predict(X_train), axis=1))
    acc_test = accuracy_score(y_test, np.argmax(model.predict(X_test), axis=1))
    
    print("n={:d} train acc={:5.3f} test acc={:5.3f}"\
         .format(n, acc_train, acc_test))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002427 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2202
[LightGBM] [Info] Number of data points in the train set: 43575, number of used features: 50
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -1.009549
[LightGBM] [Info] Start training from score -0.718597
[LightGBM] [Info] Start training from score -2.783828
[LightGBM] [Info] Start training from score -5.344701
[LightGBM] [Info] Start training from score -4.135453
[LightGBM] [Info] Start training from score -3.511350
[LightGBM] [Info] Start training from score -3.331081
n=0 train acc=0.780 test acc=0.779
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002386 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2201
[LightGBM] [Info] Number of data points in the 

## RF

In [53]:
%%time

from sklearn.ensemble import RandomForestClassifier

n_split = 10
n_X = len(y_train) // n_split
num_tree_ini = 10
num_tree_step = 10
model = RandomForestClassifier(n_estimators=num_tree_ini, warm_start=True)
for n in range(n_split) : 
    start = n * n_X
    end = (n+1) * n_X
    
    X, y = read_Xy(start, end)
    model.fit(X, y)
    acc_train = accuracy_score(y_train, model.predict(X_train))
    acc_test = accuracy_score(y_test, model.predict(X_test))
    
    print("epoch={:d} train acc={:5.3f} test acc={:5.3f}"
          .format(n, acc_train, acc_test))
    
    model.n_estimators += num_tree_step

epoch=0 train acc=0.868 test acc=0.854
epoch=1 train acc=0.891 test acc=0.872
epoch=2 train acc=0.899 test acc=0.879
epoch=3 train acc=0.902 test acc=0.883
epoch=4 train acc=0.904 test acc=0.885
epoch=5 train acc=0.905 test acc=0.886
epoch=6 train acc=0.906 test acc=0.887
epoch=7 train acc=0.906 test acc=0.887
epoch=8 train acc=0.907 test acc=0.888
epoch=9 train acc=0.907 test acc=0.889
CPU times: total: 1min 22s
Wall time: 1min 26s
